In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from pylite.test import *

In [3]:
loss_func = F.cross_entropy
def accuracy(out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()

nh,bs = 50,512

In [4]:
x_train,y_train,x_valid,y_valid = get_mnist_data()
x_train,x_valid = normalize_to(x_train,x_valid)

train_ds, valid_ds = get_ds(x_train,y_train,x_valid,y_valid)
data = TabularDataBunch(*get_dls(train_ds, valid_ds, bs))

In [5]:
x_train.mean(),x_train.std()

(tensor(3.0614e-05), tensor(1.))

In [6]:

def get_cnn_model(data):
    return nn.Sequential(
        Lambda(mnist_resize),
        nn.Conv2d( 1, 8, 5, padding=2,stride=2), nn.ReLU(), #14
        nn.Conv2d( 8,16, 3, padding=1,stride=2), nn.ReLU(), # 7
        nn.Conv2d(16,32, 3, padding=1,stride=2), nn.ReLU(), # 4
        nn.Conv2d(32,32, 3, padding=1,stride=2), nn.ReLU(), # 2
        nn.AdaptiveAvgPool2d(1),
        Lambda(flatten),
        nn.Linear(32,data.c)
    )

In [7]:
model = get_cnn_model(data)

In [12]:
model

Sequential(
  (0): Lambda()
  (1): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (2): ReLU()
  (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (4): ReLU()
  (5): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (6): ReLU()
  (7): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (8): ReLU()
  (9): AdaptiveAvgPool2d(output_size=1)
  (10): Lambda()
  (11): Linear(in_features=32, out_features=10, bias=True)
)

In [28]:
cbfs = [Recorder, partial(AvgStatsCallback,accuracy)]

In [29]:
opt = optim.SGD(model.parameters(), lr=0.4)
learn = Learner(model, opt, loss_func, data, cb_funcs=cbfs)

In [16]:
%time learn.fit(1)

train: [2.10601234375, tensor(0.2531)]
valid: [0.9262966796875, tensor(0.7364)]
Wall time: 18 s


In [17]:
device = torch.device('cuda',0)

In [20]:
device

device(type='cuda', index=0)

In [21]:
class CudaCallback(Callback):
    def begin_fit(self): self.model.cuda()
    def begin_batch(self): self.run.xb,self.run.yb = self.xb.cuda(),self.yb.cuda()

In [22]:
cbfs.append(CudaCallback)

In [23]:
model = get_cnn_model(data)

In [24]:
opt = optim.SGD(model.parameters(), lr=0.4)
learn = Learner(model, opt, loss_func, data, cb_funcs=cbfs)

In [31]:
learn.add_callbacks(cb_funcs=CudaCallback)

In [32]:
learn.cbs

In [33]:
class BatchTransformXCallback(Callback):
    _order=2
    def __init__(self, tfm): self.tfm = tfm
    def begin_batch(self): self.run.xb = self.tfm(self.xb)